In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
%config Completer.use_jedi = True

# Importando tablas

In [ ]:
df_devices = pd.read_csv('../data/devices.csv')
df_notifications = pd.read_csv('../data/notifications.csv')
df_transactions = pd.read_csv('../data/transactions.csv')
df_users = pd.read_csv('../data/users.csv')

display(df_devices.head(5), df_notifications.head(5), df_transactions.head(5), df_users.head(5))

In [ ]:
display(df_devices.info(), df_notifications.info(), df_transactions.info(), df_users.info())

# Rename

In [ ]:
## cambiar el nombre a la tabla

df_devices = df_devices.rename(columns={"string_field_0": "brand_device", "string_field_1": "user_id"})
df_notifications = df_notifications.rename(columns={'created_date': 'create_date_notification'})
df_transactions = df_transactions.rename(columns={'created_date': 'create_date_transaction'})
df_users = df_users.rename(columns={'created_date': 'create_date_user'})

display(df_devices, df_notifications, df_transactions, df_users)

In [ ]:
df_users['num_successful_referrals'].sum()

# Merge

In [ ]:
pd.set_option('display.max_columns', None)

df = df_users.merge(df_devices, on = 'user_id', how='left')
df = df.merge(df_notifications, on = "user_id", how = "left")
df = df.merge(df_transactions, on = "user_id", how = 'left')
df.head(30)

In [ ]:
df.shape

In [ ]:
df.isnull().sum().sort_values(ascending=False)

📌 1. Identifica columnas con demasiados nulos

Estas columnas tienen más de 70% de nulos, lo que generalmente amerita eliminarlas:

In [ ]:
cols_to_drop = [
    'ea_cardholderpresence',       # 295,484 nulos
    'ea_merchant_city',            # 294,220 nulos
    'ea_merchant_country',         # 294,213 nulos
    'ea_merchant_mcc'             # 294,204 nulos
]

df = df.drop(columns=cols_to_drop)
df.isnull().sum().sort_values(ascending=False)

📌 2. Imputa columnas con banderas booleanas

Estas columnas tienen ~122k nulos, pero su naturaleza es binaria (0/1). Lo más probable es que los nulos signifiquen "no lo activó", así que los puedes rellenar con 0:

In [ ]:
df['attributes_notifications_marketing_push'] = df['attributes_notifications_marketing_push'].fillna(0)
df['attributes_notifications_marketing_email'] = df['attributes_notifications_marketing_email'].fillna(0)

df.isnull().sum().sort_values(ascending=False)

📌 3. Evalúa usuarios sin actividad

Estas columnas tienen 54,604 nulos, y todas están relacionadas con transacciones:


create_date_transaction

direction

transactions_state

amount_usd

transactions_currency

transactions_type

transaction_id

👉 Eso indica que hay usuarios sin transacciones. Solo queremos usuarios activos:

In [ ]:
df = df[~df['create_date_transaction'].isnull()]
df.isnull().sum().sort_values(ascending=False)

📌 4. Imputa columnas de notificaciones
Estas tres columnas tienen 1,010 nulos:

status

create_date_notification

channel

reason

Rellenamos con 'unknown':

In [ ]:
df['status'] = df['status'].fillna('unknown')
df['create_date_notification'] = df['create_date_notification'].fillna('unknown')
df['channel'] = df['channel'].fillna('unknown')
df['reason'] = df['reason'].fillna('unknown')
df.isnull().sum().sort_values(ascending=False)

In [ ]:
display(df.shape, df.describe(), df.dtypes)

In [ ]:
df

# to_datetime

In [ ]:
df['create_date_user'] = pd.to_datetime(df['create_date_user'], errors='coerce')
df['create_date_transaction'] = pd.to_datetime(df['create_date_transaction'], errors='coerce')
df['create_date_notification'] = pd.to_datetime(df['create_date_notification'], errors='coerce')

df[['create_date_user', 'create_date_transaction', 'create_date_notification']].info()


# No tienen actividad en los últimos 30 días

In [ ]:
fecha_max = df['create_date_transaction'].max()
umbral = fecha_max - pd.Timedelta(days=30)

usuarios_activos = df[df['create_date_transaction'] > umbral]['user_id'].unique()
df_inactivos = df[~df['user_id'].isin(usuarios_activos)]

df_inactivos

# Nunca aceptaron marketing ni tienen referidos

In [ ]:
df_inactivos_marketing = df[
    (df['attributes_notifications_marketing_email'] == 0) &
    (df['attributes_notifications_marketing_push'] == 0) &
    (df['num_referrals'] == 0)
]

df_inactivos_marketing

# Distribución por edad 

In [ ]:
# Calcular la edad
from datetime import datetime
df['age'] = datetime.now().year - df['birth_year']
df

In [ ]:
# Crear rangos de edad
bins = [17, 24, 34, 44, 54, 64, 100]
labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)
df

In [ ]:
# Ver distribución (frecuencia)
df.groupby('age_group')['user_id'].nunique()

In [ ]:
# visulización
df_edad = df.groupby('age_group')['user_id'].nunique().reset_index()
df_edad.columns = ['age_group', 'unique_users']

fig = px.bar(df_edad,
             x='age_group',
             y='unique_users',
             title='Usuarios únicos por grupo de edad',
             labels={'age_group': 'Grupo de edad', 'unique_users': 'Usuarios únicos'},
             category_orders={'age_group': ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']},
             color_discrete_sequence=['#0A7C82'])  # Branding opcional
fig.update_layout(bargap=0.2)
fig.show()


# Distribución por país (Top 10)

In [ ]:
# Agrupar por país y contar usuarios únicos
df_pais = df.groupby('country')['user_id'].nunique().sort_values(ascending=False).head(10).reset_index()
df_pais.columns = ['country', 'unique_users']

df_pais

In [ ]:
# Visualizar con Plotly Express
fig = px.bar(df_pais.sort_values('unique_users', ascending=False),
             x='country',
             y='unique_users',
             title='Usuarios únicos por país',
             labels={'country': 'País', 'unique_users': 'Usuarios únicos'},
             color_discrete_sequence=['#0A7C82'])  # Color opcional
fig.update_layout(xaxis_tickangle=-45, bargap=0.2)
fig.show()


# Distribución por plan

In [ ]:
# Agrupar por plan y contar usuarios únicos
df_plan = df.groupby('plan')['user_id'].nunique().reset_index()
df_plan.columns = ['plan', 'unique_users']

df_plan

In [ ]:
# Visualizar con gráfico de barras
fig = px.bar(df_plan.sort_values('unique_users', ascending=False),
             x='plan',
             y='unique_users',
             title='Usuarios únicos por tipo de plan',
             labels={'plan': 'Tipo de plan', 'unique_users': 'Usuarios únicos'},
             color_discrete_sequence=['#0A7C82'])  # Color opcional
fig.update_layout(xaxis_tickangle=-20)
fig.show()


# Distribución de usuarios únicos según si tienen activada la función de cripto

In [ ]:
# Agrupar por la columna user_settings_crypto_unlocked y contar usuarios únicos
df_crypto = df.groupby('user_settings_crypto_unlocked')['user_id'].nunique().reset_index()
df_crypto.columns = ['crypto_unlocked', 'unique_users']

df_crypto

In [ ]:
# Visualizar con plotly.express (gráfico de pastel)

fig = px.pie(df_crypto, 
             names='crypto_unlocked', 
             values='unique_users',
             title='Distribución de usuarios por uso de funcionalidad cripto',
             labels={'crypto_unlocked': '¿Activó cripto?', 'unique_users': 'Usuarios únicos'},
             color_discrete_sequence=px.colors.sequential.Teal)

fig.update_traces(textinfo='percent+label')
fig.show()


# Distribución de usuarios únicos según su canal de notificaciones

In [ ]:
# Agrupar por canal y contar usuarios únicos
df_type_notifications = df.groupby('channel')['user_id'].nunique().reset_index()
df_type_notifications.columns = ['channel', 'unique_users']

df_type_notifications

In [ ]:
# Visualizar con plotly.express (gráfico de barras)
fig = px.bar(df_type_notifications,
             x='channel',
             y='unique_users',
             title='Usuarios únicos por canal de notificación',
             labels={'channel': 'Canal', 'unique_users': 'Usuarios únicos'},
             color='channel',
             text='unique_users')

fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()


# Distribución por número de contactos

In [ ]:
df.groupby('num_contacts')['user_id'].nunique()

In [ ]:
fig = px.histogram(
    df_users,
    x='num_contacts',
    nbins=30,  # puedes ajustar la cantidad de bins si hay mucha variación
    title='Distribución por número de contactos',
    labels={'num_contacts': 'Número de contactos'},
    color_discrete_sequence=['#00b894']  # opcional: color personalizado
)

fig.update_layout(bargap=0.1)
fig.show()

# Distribución por número de referidos exitosos

In [ ]:
display(df_users['num_successful_referrals'].dtype, df_users['num_successful_referrals'].describe())

In [ ]:
df_referrals = df_users[df_users['num_successful_referrals'] > 0]

In [ ]:
fig = px.histogram(
    df_referrals,
    x='num_successful_referrals',
    nbins=20,
    title='Distribución de usuarios con referidos exitosos (>0)',
    labels={'num_successful_referrals': 'Referidos exitosos'},
    color_discrete_sequence=['#6c5ce7']
)

fig.update_layout(bargap=0.1)
fig.show()

# Distribución por marca de dispositivo

In [ ]:
# Agrupar por marca y contar usuarios únicos
df_brand = df.groupby('brand_device')['user_id'].nunique().reset_index().sort_values(by='user_id', ascending=False)

# Renombrar columna para claridad
df_brand.columns = ['Marca del dispositivo', 'Usuarios únicos']

# Crear gráfico de barras
fig = px.bar(
    df_brand,
    x='Marca del dispositivo',
    y='Usuarios únicos',
    title='Distribución por marca de dispositivo'
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


# Distribución por tipo de transacción

In [ ]:
import plotly.express as px

# Contar número de transacciones por tipo
df_tx_type = df_transactions['transactions_type'].value_counts().reset_index()
df_tx_type.columns = ['Tipo de transacción', 'Cantidad']

# Gráfico de barras
fig = px.bar(
    df_tx_type,
    x='Tipo de transacción',
    y='Cantidad',
    title='Distribución por tipo de transacción',
    color_discrete_sequence=['#0984e3']
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


# Usuarios únicos por tipo de transacción

In [ ]:
df_tx_type_unique = df_transactions.groupby('transactions_type')['user_id'].nunique().reset_index()
df_tx_type_unique.columns = ['Tipo de transacción', 'Usuarios únicos']

fig = px.bar(
    df_tx_type_unique,
    x='Tipo de transacción',
    y='Usuarios únicos',
    title='Usuarios únicos por tipo de transacción',
    color_discrete_sequence=['#00cec9']
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


# Distribución por estado de la transacción

In [ ]:
df_state = df_transactions['transactions_state'].value_counts().reset_index()
df_state.columns = ['Estado de transacción', 'Cantidad']

fig = px.bar(
    df_state,
    x='Estado de transacción',
    y='Cantidad',
    title='Distribución por estado de la transacción',
    color_discrete_sequence=['#6c5ce7']
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()

# Usuarios únicos por estado de la transacción

In [ ]:
df_state_unique = df_transactions.groupby('transactions_state')['user_id'].nunique().reset_index()
df_state_unique.columns = ['Estado de transacción', 'Usuarios únicos']

fig = px.bar(
    df_state_unique,
    x='Estado de transacción',
    y='Usuarios únicos',
    title='Usuarios únicos por estado de la transacción',
    color_discrete_sequence=['#fd79a8']
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


# Distribución por dirección del flujo de transacción

In [ ]:
df_direction = df_transactions['direction'].value_counts().reset_index()
df_direction.columns = ['Dirección', 'Cantidad']

fig = px.bar(
    df_direction,
    x='Dirección',
    y='Cantidad',
    title='Distribución por dirección del flujo de transacción',
    color_discrete_sequence=['#00cec9']
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


# Usuarios únicos por dirección del flujo de transacción

In [ ]:
df_direction_unique = df_transactions.groupby('direction')['user_id'].nunique().reset_index()
df_direction_unique.columns = ['Dirección', 'Usuarios únicos']

fig = px.bar(
    df_direction_unique,
    x='Dirección',
    y='Usuarios únicos',
    title='Usuarios únicos por dirección del flujo de transacción',
    color_discrete_sequence=['#0984e3']
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


# Distribución por moneda de las transacciones

In [ ]:
df_currency = df_transactions['transactions_currency'].value_counts().reset_index()
df_currency.columns = ['Moneda', 'Cantidad']

fig = px.bar(
    df_currency,
    x='Moneda',
    y='Cantidad',
    title='Distribución por moneda de las transacciones',
    color_discrete_sequence=["#09bb88"]
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


In [ ]:
df_currency_unique = df.groupby('transactions_currency')['user_id'].nunique().reset_index()
df_currency_unique.columns = ['Moneda', 'Usuarios únicos']

fig = px.bar(
    df_currency_unique,
    x='Moneda',
    y='Usuarios únicos',
    title='Usuarios únicos por moneda de transacción',
    color_discrete_sequence=['#00b894']
)

fig.update_layout(xaxis_tickangle=-45)
fig.show()


# Distribución del monto (amount_usd)

In [ ]:
fig = px.histogram(
    df_transactions,
    x='amount_usd',
    nbins=100,
    title='Distribución del monto (amount_usd)',
    color_discrete_sequence=['#74b9ff']
)

fig.update_layout(xaxis_title='Monto en USD', yaxis_title='Frecuencia')
fig.show()


In [ ]:
fig = px.box(
    df_transactions,
    y='amount_usd',
    title='Distribución del monto (amount_usd) - Boxplot',
    points='all',  # para mostrar todos los puntos
    color_discrete_sequence=['#0984e3']
)

fig.update_layout(yaxis_title='Monto en USD')
fig.show()


In [ ]:
df.info()

# Distribución por país del comercio (ea_merchant_country)

In [ ]:
# Agrupamos por país y contamos el número de transacciones
country_counts = df_transactions['ea_merchant_country'].value_counts().reset_index()
country_counts.columns = ['ea_merchant_country', 'cantidad']

# Gráfico de barras
fig = px.bar(
    country_counts.head(20),  # Puedes ajustar el número si hay muchos países
    x='ea_merchant_country',
    y='cantidad',
    title='Distribución por país del comercio (ea_merchant_country)',
    color='cantidad',
    color_continuous_scale='Blues'
)

fig.update_layout(
    xaxis_title='País del comercio',
    yaxis_title='Número de transacciones',
    xaxis_tickangle=45
)

fig.show()


# Distribución por ciudad del comercio (ea_merchant_city)

In [ ]:
# Agrupar por ciudad del comercio y contar las transacciones
city_counts = df_transactions['ea_merchant_city'].value_counts().reset_index()
city_counts.columns = ['ea_merchant_city', 'cantidad']

# Gráfico de barras para las principales ciudades
fig = px.bar(
    city_counts.head(20),  # Puedes ajustar el número si hay muchas ciudades
    x='ea_merchant_city',
    y='cantidad',
    title='Distribución por ciudad del comercio (ea_merchant_city)',
    color='cantidad',
    color_continuous_scale='Oranges'
)

fig.update_layout(
    xaxis_title='Ciudad del comercio',
    yaxis_title='Número de transacciones',
    xaxis_tickangle=45
)

fig.show()


# Distribución por presencia del usuario (ea_cardholderpresence)

In [ ]:
# Contar la cantidad de transacciones por tipo de presencia
presence_counts = df_transactions['ea_cardholderpresence'].value_counts().reset_index()
presence_counts.columns = ['ea_cardholderpresence', 'cantidad']

# Gráfico de barras
fig = px.bar(
    presence_counts,
    x='ea_cardholderpresence',
    y='cantidad',
    title='Distribución por presencia del usuario (ea_cardholderpresence)',
    color='cantidad',
    color_continuous_scale='Purples'
)

fig.update_layout(
    xaxis_title='Presencia del usuario',
    yaxis_title='Número de transacciones'
)

fig.show()


# Serie de tiempo diaria

In [ ]:
# Agrupar por fecha diaria
daily_counts = df.groupby(df['create_date_transaction'].dt.date)['transaction_id'].nunique().reset_index()
daily_counts.columns = ['fecha', 'num_transacciones']

# Gráfico
fig = px.line(daily_counts, x='fecha', y='num_transacciones',
              title='Transacciones por día',
              markers=True)

fig.update_layout(xaxis_title='Fecha', yaxis_title='Cantidad de transacciones')
fig.show()


#  Serie de tiempo semanal

In [ ]:
# Agrupar por semana
df['semana'] = df['create_date_transaction'].dt.to_period('W').apply(lambda r: r.start_time)



In [ ]:
weekly_counts = df.groupby('semana')['transaction_id'].nunique().reset_index()
weekly_counts

In [ ]:
weekly_counts.columns = ['semana', 'num_transacciones']

In [ ]:
fig = px.line(weekly_counts, x='semana', y='num_transacciones',
              title='Transacciones por semana',
              markers=True)

fig.update_layout(xaxis_title='Semana', yaxis_title='Cantidad de transacciones')
fig.show()

# Serie de tiempo mensual

In [ ]:
# Agrupar por mes
df['mes'] = df['create_date_transaction'].dt.to_period('M').astype(str)
df


In [ ]:
monthly_counts = df.groupby('mes')['transaction_id'].nunique().reset_index()
monthly_counts

In [ ]:
monthly_counts.columns = ['mes', 'num_transacciones']

In [ ]:
fig = px.line(monthly_counts, x='mes', y='num_transacciones',
              title='Transacciones por mes',
              markers=True)

fig.update_layout(xaxis_title='Mes', yaxis_title='Cantidad de transacciones')
fig.show()

# Distribución por motivo de notificación

In [ ]:
df_notifications.describe()

In [ ]:
# Agrupar por motivo de notificación y contar cuántas veces aparece cada uno
reason_counts = df_notifications['reason'].value_counts().reset_index()
reason_counts.columns = ['reason', 'count']

# Crear gráfico de barras
fig = px.bar(reason_counts, 
             x='reason', 
             y='count', 
             title='Distribución por motivo de notificación',
             labels={'reason': 'Motivo de notificación', 'count': 'Número de notificaciones'},
             text='count')

fig.update_layout(xaxis_tickangle=-45)
fig.show()


# Distribución por canal de notificación

In [ ]:
# Para la distribución por status (estado de la notificación: SENT, FAILED, DELIVERED, etc.), 
# NO es necesario usar usuarios únicos, porque lo que interesa aquí es cuántas notificaciones se enviaron en cada estado, no cuántos usuarios distintos las recibieron.

# Conteo de notificaciones por canal
channel_counts = df_notifications['channel'].value_counts().reset_index()
channel_counts.columns = ['channel', 'count']

# Gráfico de barras
fig = px.bar(channel_counts,
             x='channel',
             y='count',
             title='Distribución por canal de notificación',
             labels={'channel': 'Canal', 'count': 'Cantidad'},
             text='count')

fig.update_layout(xaxis_tickangle=-45)
fig.show()



# Distribución por estado de notificación

In [ ]:
df_notifications['status'].unique()

In [ ]:
# Conteo de notificaciones por status
status_counts = df_notifications['status'].value_counts().reset_index()
status_counts.columns = ['status', 'count']

# Gráfico de barras
fig = px.bar(status_counts,
             x='status',
             y='count',
             title='Distribución por estado de notificación',
             labels={'status': 'Estado', 'count': 'Cantidad'},
             text='count')

fig.update_layout(xaxis_tickangle=-45)
fig.show()


# Notificaciones enviadas por día

In [ ]:
df_notifications.info()

In [ ]:
# Asegúrate de que la columna sea datetime
df_notifications['create_date_notification'] = pd.to_datetime(df_notifications['create_date_notification'])

In [ ]:

# Agrupar por fecha (diaria)
notificaciones_por_dia = df_notifications.groupby(df_notifications['create_date_notification'].dt.date).size().reset_index(name='count')

In [ ]:

# Graficar
fig = px.line(notificaciones_por_dia, x='create_date_notification', y='count',
              title='Notificaciones enviadas por día')
fig.show()

# Notificaciones enviadas por semana

In [ ]:
# Agrupar por semana
df_notifications['semana'] = df_notifications['create_date_notification'].dt.to_period('W').apply(lambda r: r.start_time)
df_notifications


In [ ]:
notificaciones_por_semana = df_notifications.groupby('semana').size().reset_index(name='count')
notificaciones_por_semana

In [ ]:
fig = px.line(notificaciones_por_semana, x='semana', y='count',
              title='Notificaciones enviadas por semana')
fig.show()

# Notificaciones enviadas por mes

In [ ]:
# Crear columna de año y mes
df_notifications['mes'] = df_notifications['create_date_notification'].dt.to_period('M').astype(str)
df_notifications

In [ ]:
# Agrupar por mes
notificaciones_por_mes = df_notifications.groupby('mes').size().reset_index(name='count')
notificaciones_por_mes

In [ ]:
fig = px.line(notificaciones_por_mes, x='mes', y='count',
              title='Notificaciones enviadas por mes',
              markers=True)
fig.update_layout(xaxis_title='Mes', yaxis_title='Cantidad de notificaciones')
fig.show()